### Problem sentence


You have to create two separate functions that are not allowed to comunicate between them i.e. can't use global variables.
I generate an array of size "array_size" with random integers with values less than "max_int" (values can repeat).
This array is passed to the first function that is tasked to arrange the elements in the array in whatever manner it sees fit and returns the newly ordered array to me.
I take a random element out of the array and I pass it to the second function.
The second function is tasked with finding what element I removed from the array.

Please fill in the two functions with your code.

Good luck! :)


### Import Libraries


In [109]:
import numpy as np

In [110]:
max_int: int = 10000
array_size: int = 100000

### Generate array of random numbers


In [111]:
array: np.ndarray = np.random.randint(0, max_int, size=array_size)


### Run through first program


In [112]:
def function_1(bag):
    S = np.sum(bag)
    bag = sorted(bag)
    print(list(bag))
    permutation = sorted(bag, key=lambda x: S ^ x)

    return permutation

In [113]:
array_1 = function_1(array)

[np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.int32(1), np.int32(1), np.int32(1), np.int32(1), np.int32(1), np.int32(1), np.int32(1), np.int32(2), np.int32(2), np.int32(2), np.int32(2), np.int32(2), np.int32(2), np.int32(2), np.int32(2), np.int32(2), np.int32(3), np.int32(3), np.int32(3), np.int32(3), np.int32(3), np.int32(3), np.int32(3), np.int32(4), np.int32(4), np.int32(4), np.int32(4), np.int32(4), np.int32(4), np.int32(4), np.int32(4), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(5), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(6), np.int32(7), np.int32(7), np.int32(7), np.int32(7), np.int32(7), np.int32(7), np.int32(7), np.int32(7), np.int32(8), np.int32(8)

### Remove a number from the list


In [114]:
np.random.seed()
idx = np.random.randint(0, len(array_1))
removed_element = array_1[idx]
array_1_modified = np.delete(array_1, idx)

In [115]:
removed_element

np.int32(1768)

In [116]:
array_1_modified

array([9916, 9916, 9916, ..., 6467, 6467, 6467], dtype=int32)

### Run through second program


In [117]:
from joblib import Parallel, delayed
import numba


@numba.njit(fastmath=True)
def is_sorted_ascending(arr):
    for i in range(len(arr) - 1):
        if arr[i] > arr[i + 1]:
            return False
    return True


def _process_chunk(array: np.ndarray, start_i: int, end_i: int) -> int:
    truncated_perm = np.ascontiguousarray(array.ravel(), dtype=np.uint64)
    S_trunc = truncated_perm.sum(dtype=np.uint64)
    for g in range(start_i, end_i):
        S_candidate = S_trunc + g
        keys = S_candidate ^ truncated_perm

        if is_sorted_ascending(keys):
            return g

    return -1


def function_2(
    array: np.ndarray, max_int: int, chunk_size: int = 10000, n_jobs: int = -1
) -> int:
    array = np.ascontiguousarray(array, dtype=np.int64)

    n_chunks = (max_int // chunk_size) + (1 if max_int % chunk_size else 0)

    tasks = (
        delayed(_process_chunk)(
            array, i * chunk_size, min((i + 1) * chunk_size, max_int)
        )
        for i in range(n_chunks)
    )

    results = Parallel(n_jobs=n_jobs)(tasks)

    valid_is = [res for res in results if res != -1]

    return valid_is[0] if valid_is else -1

In [118]:
answer = function_2(array_1_modified, max_int)

In [119]:
(
    answer,
    removed_element,
    f"difference: {abs(answer - removed_element) / np.mean([answer, removed_element]) * 100}%",
)

(1768, np.int32(1768), 'difference: 0.0%')